In [69]:
import gc
import time
from datetime import datetime
import warnings
warnings.simplefilter(action = 'ignore')
import numpy as np
import pandas as pd
import gc
import time
from datetime import datetime
import warnings
warnings.simplefilter(action = 'ignore')
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from scipy.stats import mannwhitneyu

In [63]:
import numpy as np
import pandas as pd
train = pd.read_csv('train.csv', index_col = 'id')
train.head()

,target,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [64]:
target = train['target']
train.drop('target', axis = 1, inplace = True)
target.value_counts()

1.0    160
0.0     90
Name: target, dtype: int64

In [65]:
test = pd.read_csv('test.csv', index_col = 'id')
test.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
250,0.500,-1.033,-1.595,0.309,-0.714,0.502,0.535,-0.129,-0.687,1.291,...,-0.088,-2.628,-0.845,2.078,-0.277,2.132,0.609,-0.104,0.312,0.979
251,0.776,0.914,-0.494,1.347,-0.867,0.480,0.578,-0.313,0.203,1.356,...,-0.683,-0.066,0.025,0.606,-0.353,-1.133,-3.138,0.281,-0.625,-0.761
252,1.750,0.509,-0.057,0.835,-0.476,1.428,-0.701,-2.009,-1.378,0.167,...,-0.094,0.351,-0.607,-0.737,-0.031,0.701,0.976,0.135,-1.327,2.463
253,-0.556,-1.855,-0.682,0.578,1.592,0.512,-1.419,0.722,0.511,0.567,...,-0.336,-0.787,0.255,-0.031,-0.836,0.916,2.411,1.053,-1.601,-1.529
254,0.754,-0.245,1.173,-1.623,0.009,0.370,0.781,-1.763,-1.432,-0.930,...,2.184,-1.090,0.216,1.186,-0.143,0.322,-0.068,-0.156,-1.153,0.825


In [66]:
def save_submit(test_, clfs_, filename):
    subm = pd.DataFrame(np.zeros(test_.shape[0]), index = test_.index, columns = ['target'])
    for clf in clfs_:
        subm['target'] += clf.predict_proba(test_)[:, 1]
    subm['target'] /= len(clfs_)
    subm = subm.reset_index()
    subm.columns = ['id', 'target']
    subm.to_csv(filename, index = False)

In [67]:
scores = pd.DataFrame(columns = ['auc', 'acc', 'loss', 'tn', 'fn', 'fp', 'tp'])

In [70]:
mw = pd.DataFrame(index = ['stat', 'p'])
for c in train.columns:
    mw[c] = mannwhitneyu(train[c], test[c])
mw = mw.T
mw[mw['p'] < .01].shape

(6, 2)

In [71]:
bad_features = list(mw[mw['p'] < .01].index)
len(bad_features)

6

In [100]:
params = {}
params['random_state'] = 0
params['n_jobs'] = -1
params['C'] = 0.2
params['penalty'] = 'l1'
params['class_weight'] = 'balance'
params['solver'] = 'saga'

In [107]:
def logreg_cross_validation(train_, target_, params,
                            num_folds = 7, repeats = 20, rs = 0):
    
    print(params)
    
    clfs = []
    folds = RepeatedStratifiedKFold(n_splits = num_folds, n_repeats = repeats, random_state = rs)
    
    valid_pred = pd.DataFrame(index = train_.index)
    
    # Cross-validation cycle
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(target_, target_)):
        print('--- Fold {} started at {}'.format(n_fold, time.ctime()))
        
        train_x, train_y = train_.iloc[train_idx], target_.iloc[train_idx]
        valid_x, valid_y = train_.iloc[valid_idx], target_.iloc[valid_idx]
        
        clf = LogisticRegression(**params)
        clf.fit(train_x, train_y)
    
        clfs.append(clf)

        predict = clf.predict_proba(valid_x)[:, 1]
    
        tn, fp, fn, tp = confusion_matrix(valid_y, (predict >= .5) * 1).ravel()
        auc = roc_auc_score(valid_y, predict)
        acc = accuracy_score(valid_y, (predict >= .5) * 1)
        loss = log_loss(valid_y, predict)
        print('TN =', tn, 'FN =', fn, 'FP =', fp, 'TP =', tp)
        print('AUC = ', auc, 'Loss =', loss, 'Acc =', acc)
        
        valid_pred[n_fold] = pd.Series(predict, index = valid_x.index)
        del train_x, train_y, valid_x, valid_y, predict
        gc.collect()

    return clfs, valid_pred

In [108]:
clfs, pred = logreg_cross_validation(train.drop(bad_features, axis = 1), target, params)

{'random_state': 0, 'n_jobs': -1, 'C': 0.2, 'penalty': 'l1', 'class_weight': 'balance', 'solver': 'saga'}
--- Fold 0 started at Sun Jun 23 22:43:47 2019
TN = 9 FN = 4 FP = 4 TP = 19
AUC =  0.8060200668896321 Loss = 0.499175471131754 Acc = 0.7777777777777778
--- Fold 1 started at Sun Jun 23 22:43:48 2019
TN = 6 FN = 3 FP = 7 TP = 20
AUC =  0.8060200668896321 Loss = 0.5095779399268414 Acc = 0.7222222222222222
--- Fold 2 started at Sun Jun 23 22:43:48 2019
TN = 6 FN = 4 FP = 7 TP = 19
AUC =  0.7959866220735786 Loss = 0.515207878565743 Acc = 0.6944444444444444
--- Fold 3 started at Sun Jun 23 22:43:48 2019
TN = 6 FN = 1 FP = 7 TP = 22
AUC =  0.7224080267558528 Loss = 0.5860917469607038 Acc = 0.7777777777777778
--- Fold 4 started at Sun Jun 23 22:43:48 2019
TN = 6 FN = 5 FP = 7 TP = 18
AUC =  0.802675585284281 Loss = 0.5382102190192055 Acc = 0.6666666666666666
--- Fold 5 started at Sun Jun 23 22:43:48 2019
TN = 4 FN = 5 FP = 9 TP = 18
AUC =  0.6655518394648829 Loss = 0.6632728172386193 Acc 

TN = 6 FN = 4 FP = 6 TP = 18
AUC =  0.7462121212121212 Loss = 0.5485092792868104 Acc = 0.7058823529411765
--- Fold 56 started at Sun Jun 23 22:43:55 2019
TN = 6 FN = 5 FP = 7 TP = 18
AUC =  0.7993311036789298 Loss = 0.5082560571997717 Acc = 0.6666666666666666
--- Fold 57 started at Sun Jun 23 22:43:55 2019
TN = 5 FN = 5 FP = 8 TP = 18
AUC =  0.7892976588628763 Loss = 0.5061602943045229 Acc = 0.6388888888888888
--- Fold 58 started at Sun Jun 23 22:43:55 2019
TN = 9 FN = 3 FP = 4 TP = 20
AUC =  0.903010033444816 Loss = 0.40156423746926795 Acc = 0.8055555555555556
--- Fold 59 started at Sun Jun 23 22:43:55 2019
TN = 8 FN = 5 FP = 5 TP = 18
AUC =  0.7792642140468227 Loss = 0.5577548258300289 Acc = 0.7222222222222222
--- Fold 60 started at Sun Jun 23 22:43:55 2019
TN = 6 FN = 5 FP = 7 TP = 18
AUC =  0.7123745819397993 Loss = 0.6396374793139816 Acc = 0.6666666666666666
--- Fold 61 started at Sun Jun 23 22:43:55 2019
TN = 7 FN = 0 FP = 6 TP = 23
AUC =  0.8327759197324415 Loss = 0.475157242674

TN = 6 FN = 3 FP = 7 TP = 20
AUC =  0.745819397993311 Loss = 0.5706037044566419 Acc = 0.7222222222222222
--- Fold 110 started at Sun Jun 23 22:44:02 2019
TN = 7 FN = 1 FP = 6 TP = 22
AUC =  0.8662207357859532 Loss = 0.4530330133909772 Acc = 0.8055555555555556
--- Fold 111 started at Sun Jun 23 22:44:02 2019
TN = 5 FN = 2 FP = 7 TP = 20
AUC =  0.7348484848484849 Loss = 0.560643046763943 Acc = 0.7352941176470589
--- Fold 112 started at Sun Jun 23 22:44:02 2019
TN = 7 FN = 0 FP = 6 TP = 23
AUC =  0.8595317725752508 Loss = 0.42214078675664185 Acc = 0.8333333333333334
--- Fold 113 started at Sun Jun 23 22:44:02 2019
TN = 4 FN = 4 FP = 9 TP = 19
AUC =  0.6254180602006689 Loss = 0.7234155640665019 Acc = 0.6388888888888888
--- Fold 114 started at Sun Jun 23 22:44:02 2019
TN = 5 FN = 5 FP = 8 TP = 18
AUC =  0.7290969899665551 Loss = 0.5808098268121911 Acc = 0.6388888888888888
--- Fold 115 started at Sun Jun 23 22:44:03 2019
TN = 7 FN = 3 FP = 6 TP = 20
AUC =  0.862876254180602 Loss = 0.45513883

In [109]:
len(clfs)

140

In [103]:
pred_mean = pred.mean(axis = 1)

In [104]:
scores = scores.T
tn, fp, fn, tp = confusion_matrix(target, (pred_mean >= .5) * 1).ravel()
scores['logreg'] = [
                 roc_auc_score(target, pred_mean), 
                 accuracy_score(target, (pred_mean >= .5) * 1), 
                 log_loss(target, pred_mean),
                 tn, fn, fp, tp]

scores = scores.T
scores

,auc,acc,loss,tn,fn,fp,tp
logreg,0.812639,0.752,0.49836,47.0,19.0,43.0,141.0


In [89]:
score_auc = scores.loc['logreg', 'auc']
score_acc = scores.loc['logreg', 'acc']
score_loss = scores.loc['logreg', 'loss']
print(score_auc, score_acc, score_loss)

0.8138888888888889 0.736 0.5059652809887581


In [90]:
ver = 'logreg'
loc_ver = 'v1'
filename = 'subm_{}_{}_{:.4f}_{:.4f}_{:.4f}_{}.csv'.format(ver, loc_ver, score_auc, score_acc, score_loss,
                                                        datetime.now().strftime('%Y-%m-%d'))
print(filename)
save_submit(test.drop(bad_features, axis = 1), clfs, filename)

subm_logreg_v1_0.8139_0.7360_0.5060_2019-06-23.csv
